In [1]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

# Load your dataset
df = pd.read_csv("train.csv")  # Replace with your actual file path

# Remove rows with NaN values in 'crimeaditionalinfo'
df = df.dropna(subset=["crimeaditionalinfo"])

# Encode categories and subcategories
category_encoder = LabelEncoder()
sub_category_encoder = LabelEncoder()

df["category_label"] = category_encoder.fit_transform(df["category"])
df["sub_category_label"] = sub_category_encoder.fit_transform(df["sub_category"])

# Combine labels for a single classification task if predicting both category and subcategory together
df["combined_label"] = df["category_label"] * 100 + df["sub_category_label"]

# Rename columns for Hugging Face Dataset
df = df.rename(columns={"crimeaditionalinfo": "text", "combined_label": "label"})

# Convert to a Hugging Face Dataset
dataset = Dataset.from_pandas(df[["text", "label"]])
dataset = dataset.train_test_split(test_size=0.2)


In [8]:
df

,category,sub_category,text,category_label,sub_category_label,label
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...,9,4,904
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...,7,15,715
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...,8,21,821
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...,9,22,922
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...,7,15,715
...,...,...,...,...,...,...
93681,Online Financial Fraud,Internet Banking Related Fraud,Identity theft Smishing SMS Fraud CreditDeb...,7,18,718
93682,Online Financial Fraud,EWallet Related Fraud,RECEIVED CALL FROM NUMBER ASKING ABOUT phone ...,7,12,712
93683,Online Financial Fraud,UPI Related Frauds,Cyber Stalking Blackmailing PhoneSMSVOIP C...,7,32,732
93684,Online and Social Media Related Crime,Online Matrimonial Fraud,Call karke bola ki aapka lotary laga ha aru AC...,9,23,923


In [9]:
df.isnull().sum()

,0
category,0
sub_category,6591
text,0
category_label,0
sub_category_label,0
label,0


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 74932
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 18733
    })
})

In [11]:
dataset['test'][0]

{'text': 'Dear Sir i applied for  lakhs loan after seeing a message in facebook they called me back and asked to credit back some amount in order for approval of my loan Received calls and ids from these numbers   these people are in touch with email also i credited back  and  after sending these amount they asked me for tax amount of  so i got a doubt on this and stopped responding to them they started calling me repeatedely but i didnt responded\r\nSirwithin Minutes of time i lost  rupees\r\nplease do the needfull as early as possible \r\nThank You Sir',
 'label': 701,
 '__index_level_0__': 89525}

In [13]:
# Load the tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_labels = df["label"].nunique()
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/74932 [00:00<?, ? examples/s]

Map:   0%|          | 0/18733 [00:00<?, ? examples/s]

In [19]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

# Start training
trainer.train()


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [18]:
!export CUDA_LAUNCH_BLOCKING=1
